In [1]:
import torch
import torchvision
import torchvision.transforms as transforms

torch.set_printoptions(linewidth=120)
torch.set_grad_enabled(True)

In [2]:
 train_set = torchvision.datasets.FashionMNIST(    # Extract
     root = '.data/FashionMNIST',
     train = True,
     download = True,
     transform = transforms.Compose([             # Transform
         transforms.ToTensor()
     ])
 )

In [3]:
import torch.nn as nn 
import torch.nn.functional as F

class Network(nn.Module):
    def __init__(self):
        super(Network,self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5)
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=12, kernel_size=5)
        
        self.fc1 = nn.Linear(in_features=12*4*4 , out_features=120)
        self.fc2 = nn.Linear(in_features=120, out_features=60)
        self.out = nn.Linear(in_features=60, out_features=10)
        
    def forward(self, t):
        # (1) input layer
        t = t
        
        # (2) hiddden conv layer
        t = self.conv1(t)
        t = F.relu(t)
        t = F.max_pool2d(t,kernel_size=2,stride=2)
        
        # (3) hidden conv layer
        t = self.conv2(t)
        t = F.relu(t)
        t = F.max_pool2d(t,kernel_size=2,stride=2)
        
        # (4) hidden linear layer
        t = t.reshape(-1,12*4*4)
        t = self.fc1(t)
        t = F.relu(t)
        
        # (5) hidden linear layer
        t = self.fc2(t)
        t = F.relu(t)
        
        # (6) output layer
        t = self.out(t)
        # t = F.softmax(t,dim=1) (since loss is cross_entropy which automatically uses softmax)
        
        return t

In [4]:
def get_num_correct(preds,labels):
    return preds.argmax(dim=1).eq(labels).sum().item()

In [7]:
network = Network()

train_loader = torch.utils.data.DataLoader(train_set, batch_size = 100) #Load
optimizer = torch.optim.Adam(network.parameters(), lr=0.01)

for epoch in range(5):
    
    total_loss = 0
    total_correct = 0
    for batch in train_loader:
        images, labels = batch

        preds = network(images)
        loss = F.cross_entropy(preds, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        total_correct += get_num_correct(preds, labels)

    print("epoch: ", epoch ,"total_correct: ", total_correct, "loss: ",total_loss)    

epoch:  0 total_correct:  46361 loss:  360.227274030447
epoch:  1 total_correct:  50895 loss:  246.47023908793926
epoch:  2 total_correct:  51638 loss:  226.43659210205078
epoch:  3 total_correct:  51860 loss:  219.97004471719265
epoch:  4 total_correct:  52120 loss:  214.93527199327946


In [8]:
total_correct / len(train_set)

0.8686666666666667